In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import joypy
from datetime import datetime
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import matplotlib.ticker as mtick


In [37]:
df = pd.read_csv("trippub.csv")
df = df[['HOUSEID','PERSONID','HHSTATE','TRPTRANS','TRPMILES','WTTRDFIN','TRAVDAY','TRIPPURP','HH_CBSA','STRTTIME']]

day = { 1 : 'Sunday', 2: 'Weekday', 3: 'Weekday', 4: 'Weekday', 5: "Weekday", 6: "Weekday", 7: "Saturday"}
day = pd.Series(day).to_frame('Day')
day = day.reset_index().rename(columns = {'index' : 'TRAVDAY'})

PURPOSE = { 'HBW' : "HBW", "HBO" : "HBO", "HBOSHOP": "HBO", "HBOSOCREC": "HBO", "NHB" : "NHB" }
PURPOSE = pd.Series(PURPOSE).to_frame('PURP')
PURPOSE = PURPOSE.reset_index().rename(columns = {'index' : 'TRIPPURP'})

df = pd.merge(df, day, on = 'TRAVDAY')
df = df.loc[(df['TRIPPURP'] != '-9'),]

df = pd.merge(df, PURPOSE, on = 'TRIPPURP')

df['TripDist'] = df['TRPMILES'] * df['WTTRDFIN']

df['Time'] = df['STRTTIME'].apply(lambda num : format(num,'04d'))
df['DateTime'] = df['Time'].apply(lambda num : datetime.strptime(num, "%H%M").strftime("%I:%M %p"))
df['Hour'] = pd.to_datetime(df['DateTime'], format='%I:%M %p')
df['NearestHour'] = df['Hour'].dt.round(freq='60min')


national_bike = df.loc[(df["TRPTRANS"] == 2),]
national_walk = df.loc[df["TRPTRANS"] == 1,]
weights_bike = national_bike['WTTRDFIN'].sum()

In [27]:
df.head()
df.dtypes

HOUSEID       int64
PERSONID      int64
HHSTATE      object
TRPTRANS      int64
TRPMILES    float64
WTTRDFIN    float64
TRAVDAY       int64
TRIPPURP     object
HH_CBSA      object
STRTTIME      int64
Day          object
PURP         object
TripDist    float64
dtype: object

In [39]:
# National Average Bike Trip Distance 

avg_dist_nat_bike = national_bike.groupby('Day')['TripDist','WTTRDFIN'].sum()
avg_dist_nat_bike = pd.DataFrame(avg_dist_nat_bike)
avg_dist_nat_bike['AvgDist'] = avg_dist_nat_bike['TripDist']/avg_dist_nat_bike['WTTRDFIN']
avg_dist_nat_bike

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,TripDist,WTTRDFIN,AvgDist
Day,,,
Saturday,1.124916e+09,2.773284e+08,4.056261
Sunday,6.670968e+08,1.719340e+08,3.879959
Weekday,3.740563e+09,1.576857e+09,2.372164


In [40]:
# National Average Walk Trip Distance 

avg_dist_nat_walk = national_walk.groupby('Day')['TripDist','WTTRDFIN'].sum()
avg_dist_nat_walk = pd.DataFrame(avg_dist_nat_walk)
avg_dist_nat_walk['AvgDist'] = avg_dist_nat_walk['TripDist']/avg_dist_nat_walk['WTTRDFIN']
avg_dist_nat_walk

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,TripDist,WTTRDFIN,AvgDist
Day,,,
Saturday,1.676141e+09,2.605913e+09,0.643207
Sunday,1.683127e+09,2.357477e+09,0.713953
Weekday,1.816740e+10,1.925264e+10,0.943632


In [41]:
# National Average Bike Trip Distance by Tour Purpose

avg_dist_nat_bike = national_bike.groupby(['Day','PURP'])['TripDist','WTTRDFIN'].sum()
avg_dist_nat_bike = pd.DataFrame(avg_dist_nat_bike)
avg_dist_nat_bike.reset_index()

avg_dist_nat_bike['AvgDist'] = avg_dist_nat_bike['TripDist']/avg_dist_nat_bike['WTTRDFIN']

avg_dist_nat_bike.pivot_table(index = ['Day'], values = 'AvgDist', columns = 'PURP')

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


PURP,HBO,HBW,NHB
Day,,,
Saturday,2.635886,1.991578,5.113414
Sunday,5.871699,2.924829,3.208333
Weekday,2.213762,2.794740,2.068974


In [42]:
# National Average Walk Trip Distance 
avg_dist_nat_walk = national_walk.groupby(['Day','PURP'])['TripDist','WTTRDFIN'].mean()
avg_dist_nat_walk = pd.DataFrame(avg_dist_nat_walk)
avg_dist_nat_walk.reset_index()

avg_dist_nat_walk['AvgDist'] = avg_dist_nat_walk['TripDist']/avg_dist_nat_walk['WTTRDFIN']

avg_dist_nat_walk.pivot_table(index = ['Day'], values = 'AvgDist', columns = 'PURP')

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


PURP,HBO,HBW,NHB
Day,,,
Saturday,0.817154,0.665986,0.513202
Sunday,0.901161,0.660000,0.530508
Weekday,0.885562,1.551666,0.894429


In [ ]:
#Time of Day

In [43]:
#Time of Day Bike National

avg_dist_nat_bike = national_bike.groupby(['NearestHour','PURP'])['TripDist','WTTRDFIN'].sum()
avg_dist_nat_bike = pd.DataFrame(avg_dist_nat_bike)
avg_dist_nat_bike.reset_index()

avg_dist_nat_bike['AvgDist'] = avg_dist_nat_bike['TripDist']/avg_dist_nat_bike['WTTRDFIN']

avg_dist_nat_bike.pivot_table(index = ['NearestHour'], values = 'AvgDist', columns = 'PURP')

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


PURP,HBO,HBW,NHB
NearestHour,,,
1900-01-01 00:00:00,NaN,0.977058,1.514000
1900-01-01 01:00:00,0.432000,0.993076,NaN
1900-01-01 02:00:00,NaN,2.321197,1.050384
1900-01-01 04:00:00,5.095175,1.191112,0.304000
1900-01-01 05:00:00,12.629936,1.091895,1.826577
1900-01-01 06:00:00,10.843460,4.338426,2.452367
1900-01-01 07:00:00,2.137506,2.870058,8.431268
1900-01-01 08:00:00,1.650946,2.763413,3.238734
1900-01-01 09:00:00,2.382128,2.641911,2.976190


In [45]:
#Time of Day Bike National

avg_dist_nat_bike = national_bike.groupby(['Day','NearestHour','PURP'])['TripDist','WTTRDFIN'].sum()
avg_dist_nat_bike = pd.DataFrame(avg_dist_nat_bike)
avg_dist_nat_bike.reset_index()

avg_dist_nat_bike['AvgDist'] = avg_dist_nat_bike['TripDist']/avg_dist_nat_bike['WTTRDFIN']

avg_dist_nat_bike.pivot_table(index = ['NearestHour'], values = 'AvgDist', columns = ['PURP', 'Day'])

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


PURP                       HBO                             HBW            \
Day                   Saturday     Sunday    Weekday  Saturday    Sunday   
NearestHour                                                                
1900-01-01 00:00:00        NaN        NaN        NaN       NaN       NaN   
1900-01-01 01:00:00        NaN        NaN   0.432000       NaN       NaN   
1900-01-01 02:00:00        NaN        NaN        NaN  2.321197       NaN   
1900-01-01 04:00:00  82.000000        NaN   4.302641       NaN       NaN   
1900-01-01 05:00:00        NaN        NaN  12.629936  1.471107  0.815000   
1900-01-01 06:00:00   8.131986  13.000000  11.052180  0.260000  1.164000   
1900-01-01 07:00:00   1.980186  16.000000   2.091476  3.266000  5.067919   
1900-01-01 08:00:00   1.489229   0.797000   1.666058  2.703725  1.180167   
1900-01-01 09:00:00   1.461941  14.863789   1.357750  1.610000  0.903968   
1900-01-01 10:00:00   2.252891  11.692580   3.213068  3.186994  1.586000   
1900-01-01 11:00:00   0.533549   3.230608   1.295465       NaN  3.709005   
1900-01-01 12:00:00   3.447602   7.691250   2.886296  2.199568  0.816898   
1900-01-01 13:00:00   0.796882   4.907779   1.972315       NaN  1.012000   
1900-01-01 14:00:00   3.225123   4.187379   1.949046  1.675446  1.674574   
1900-01-01 15:00:00   2.280993   1.094631   1.528372  1.330982  3.998409   
1900-01-01 16:00:00   4.263803   3.754321   2.171319  0.274817  3.062022   
1900-01-01 17:00:00   0.954733   1.829231   1.811702  2.333875  0.933965   
1900-01-01 18:00:00   1.628139   2.782077   2.489123  0.773293  3.619319   
1900-01-01 19:00:00   3.025764   1.608383   1.373767  2.683979       NaN   
1900-01-01 20:00:00   5.508147   0.546458   2.188050  2.742686  2.315209   
1900-01-01 21:00:00   1.220818   2.125000   2.343251       NaN  2.473000   
1900-01-01 22:00:00        NaN   0.866000   0.509839  3.743647       NaN   
1900-01-01 23:00:00   1.242000        NaN        NaN  0.376000       NaN   
1900-01-02 00:00:00        NaN   0.187000        NaN       NaN  5.329525   

PURP                                 NHB                        
Day                   Weekday   Saturday     Sunday    Weekday  
NearestHour                                                     
1900-01-01 00:00:00  0.977058        NaN        NaN   1.514000  
1900-01-01 01:00:00  0.993076        NaN        NaN        NaN  
1900-01-01 02:00:00       NaN        NaN        NaN   1.050384  
1900-01-01 04:00:00  1.191112        NaN        NaN   0.304000  
1900-01-01 05:00:00  1.185827        NaN   1.781000   2.063818  
1900-01-01 06:00:00  4.510542        NaN   1.781000   2.960983  
1900-01-01 07:00:00  2.692359  16.413751   4.697037   3.369138  
1900-01-01 08:00:00  2.810070   5.475450   4.262797   2.681790  
1900-01-01 09:00:00  2.773097   6.541877   6.996878   1.759282  
1900-01-01 10:00:00  1.804423   1.827873  15.454208   4.321947  
1900-01-01 11:00:00  2.516517   4.011251   1.189338   1.520342  
1900-01-01 12:00:00  1.544086   8.047432   4.500412   1.676610  
1900-01-01 13:00:00  2.161661   4.174328   1.038029   2.009409  
1900-01-01 14:00:00  2.131296   7.620966   2.624041   2.556304  
1900-01-01 15:00:00  2.375856  12.721391   2.025272   1.532434  
1900-01-01 16:00:00  3.844023   1.739053   1.763974   1.493463  
1900-01-01 17:00:00  2.542989   0.936756   2.017828   1.349210  
1900-01-01 18:00:00  3.723604   1.575611   0.922801   1.918735  
1900-01-01 19:00:00  1.788144   8.938531   1.217371   2.451325  
1900-01-01 20:00:00  2.232177   6.284865   0.287273   1.662962  
1900-01-01 21:00:00  1.283150   1.401000   0.909829   1.500323  
1900-01-01 22:00:00  1.327592   3.755000   0.601000   6.317111  
1900-01-01 23:00:00  2.187506        NaN        NaN   0.873930  
1900-01-02 00:00:00  4.557749   1.021000        NaN  28.166000

In [47]:
# Adding Percentage of Weights
avg_dist_nat_bike = national_bike.groupby(['Day','NearestHour','PURP'])['WTTRDFIN'].sum()
avg_dist_nat_bike = pd.DataFrame(avg_dist_nat_bike)
avg_dist_nat_bike.reset_index()

avg_dist_nat_bike['WeightedTrips'] = avg_dist_nat_bike['WTTRDFIN']

avg_dist_nat_bike.pivot_table(index = ['NearestHour'], values = 'WeightedTrips', columns = ['PURP', 'Day'])

PURP                          HBO                                       HBW  \
Day                      Saturday        Sunday       Weekday      Saturday   
NearestHour                                                                   
1900-01-01 00:00:00           NaN           NaN           NaN           NaN   
1900-01-01 01:00:00           NaN           NaN  6.755614e+05           NaN   
1900-01-01 02:00:00           NaN           NaN           NaN  6.866840e+05   
1900-01-01 04:00:00  1.968992e+04           NaN  1.910643e+06           NaN   
1900-01-01 05:00:00           NaN           NaN  2.311915e+06  1.283734e+05   
1900-01-01 06:00:00  1.042427e+06  4.459378e+04  1.308139e+07  2.623081e+05   
1900-01-01 07:00:00  4.371589e+06  1.162770e+05  2.007747e+07  2.944773e+06   
1900-01-01 08:00:00  8.442797e+06  2.098055e+05  1.022049e+08  2.273762e+06   
1900-01-01 09:00:00  5.507711e+06  2.628861e+06  2.708418e+07  9.196161e+05   
1900-01-01 10:00:00  1.220844e+07  9.738009e+06  3.310829e+07  4.049722e+06   
1900-01-01 11:00:00  6.009500e+05  4.339311e+05  3.067107e+07           NaN   
1900-01-01 12:00:00  1.081646e+07  8.744914e+06  4.253415e+07  1.359214e+06   
1900-01-01 13:00:00  6.206044e+05  2.305501e+06  2.492636e+07           NaN   
1900-01-01 14:00:00  2.419582e+06  4.040102e+06  5.539672e+07  3.364871e+05   
1900-01-01 15:00:00  3.529124e+06  5.315328e+05  4.432110e+07  2.091948e+06   
1900-01-01 16:00:00  9.382417e+06  1.573689e+06  5.232352e+07  5.363417e+05   
1900-01-01 17:00:00  4.092922e+06  4.002842e+06  3.313220e+07  3.607493e+06   
1900-01-01 18:00:00  3.681895e+06  3.416788e+06  3.171919e+07  3.915344e+06   
1900-01-01 19:00:00  1.540475e+07  7.340167e+05  2.088258e+07  5.901693e+05   
1900-01-01 20:00:00  2.590898e+05  8.291583e+06  1.086874e+07  6.464428e+05   
1900-01-01 21:00:00  1.770114e+05  1.245420e+06  8.155107e+05           NaN   
1900-01-01 22:00:00           NaN  1.394446e+04  1.104095e+07  1.431737e+05   
1900-01-01 23:00:00  9.567636e+05           NaN           NaN  3.126531e+06   
1900-01-02 00:00:00           NaN  3.277233e+05           NaN           NaN   

PURP                                                      NHB                \
Day                        Sunday       Weekday      Saturday        Sunday   
NearestHour                                                                   
1900-01-01 00:00:00           NaN  8.689072e+05           NaN           NaN   
1900-01-01 01:00:00           NaN  4.508398e+06           NaN           NaN   
1900-01-01 02:00:00           NaN           NaN           NaN           NaN   
1900-01-01 04:00:00           NaN  6.161682e+05           NaN           NaN   
1900-01-01 05:00:00  5.013297e+05  9.595826e+05           NaN  1.667954e+06   
1900-01-01 06:00:00  1.495348e+06  3.379521e+07           NaN  1.667954e+06   
1900-01-01 07:00:00  2.297011e+06  3.497188e+07  2.684518e+06  8.672453e+05   
1900-01-01 08:00:00  2.529343e+06  8.874029e+07  6.290869e+06  3.162095e+06   
1900-01-01 09:00:00  3.970163e+06  5.983008e+07  6.470572e+06  3.930145e+06   
1900-01-01 10:00:00  1.232277e+05  1.905840e+07  1.143793e+07  3.293597e+06   
1900-01-01 11:00:00  7.825562e+06  1.502337e+07  1.860405e+07  2.144342e+06   
1900-01-01 12:00:00  5.195931e+05  1.066073e+07  1.986269e+07  1.606374e+07   
1900-01-01 13:00:00  1.533850e+06  1.611408e+07  3.663661e+06  3.330455e+06   
1900-01-01 14:00:00  5.436859e+06  2.367862e+07  2.670426e+07  8.192141e+06   
1900-01-01 15:00:00  7.601800e+06  1.737171e+07  9.031876e+06  7.123112e+06   
1900-01-01 16:00:00  1.896137e+06  4.674004e+07  8.364133e+06  3.335239e+06   
1900-01-01 17:00:00  2.238076e+06  5.815390e+07  1.875398e+07  4.615735e+06   
1900-01-01 18:00:00  3.229029e+06  5.535343e+07  2.284447e+07  2.295507e+06   
1900-01-01 19:00:00           NaN  2.790522e+07  4.232032e+06  3.824423e+06   
1900-01-01 20:00:00  8.078103e+05  1.055851e+07  2.893999e+06  5.452317e+06   
1900-01-01 21:00:00  8.974195e+05  2.961262e+06  

In [51]:
# Adding Percentage of UNWeights
avg_dist_nat_bike = national_bike.groupby(['Day','NearestHour','PURP']).size().reset_index(name='counts')
avg_dist_nat_bike = pd.DataFrame(avg_dist_nat_bike)
avg_dist_nat_bike.reset_index()

avg_dist_nat_bike['UNWeightedTrips'] = avg_dist_nat_bike['counts']

avg_dist_nat_bike.pivot_table(index = ['NearestHour'], values = 'UNWeightedTrips', columns = ['PURP', 'Day'])

PURP                     HBO                     HBW                     NHB  \
Day                 Saturday Sunday Weekday Saturday Sunday Weekday Saturday   
NearestHour                                                                    
1900-01-01 00:00:00      NaN    NaN     NaN      NaN    NaN     2.0      NaN   
1900-01-01 01:00:00      NaN    NaN     1.0      NaN    NaN     3.0      NaN   
1900-01-01 02:00:00      NaN    NaN     NaN      2.0    NaN     NaN      NaN   
1900-01-01 04:00:00      1.0    NaN     3.0      NaN    NaN     3.0      NaN   
1900-01-01 05:00:00      NaN    NaN     5.0      2.0    1.0     8.0      NaN   
1900-01-01 06:00:00      2.0    1.0    29.0      1.0    1.0    56.0      NaN   
1900-01-01 07:00:00      3.0    1.0    90.0      1.0    4.0    88.0      4.0   
1900-01-01 08:00:00     16.0    1.0   272.0      4.0    7.0   184.0     14.0   
1900-01-01 09:00:00     21.0    7.0    69.0      1.0    3.0    95.0     15.0   
1900-01-01 10:00:00     28.0   13.0   100.0      3.0    1.0    55.0     27.0   
1900-01-01 11:00:00      6.0    3.0    60.0      NaN    7.0    29.0     30.0   
1900-01-01 12:00:00      9.0   10.0    80.0      2.0    2.0    54.0     26.0   
1900-01-01 13:00:00      4.0    7.0    64.0      NaN    1.0    34.0     14.0   
1900-01-01 14:00:00     12.0   15.0   118.0      3.0    4.0    36.0     26.0   
1900-01-01 15:00:00      7.0    4.0   158.0      3.0    5.0    27.0      9.0   
1900-01-01 16:00:00     10.0   11.0   157.0      2.0    3.0    91.0     19.0   
1900-01-01 17:00:00      6.0    9.0    79.0      4.0    2.0   137.0     11.0   
1900-01-01 18:00:00      5.0    7.0    93.0      2.0    3.0   118.0     18.0   
1900-01-01 19:00:00      8.0    3.0    43.0      2.0    NaN    38.0      3.0   
1900-01-01 20:00:00      3.0    7.0    40.0      2.0    4.0    27.0      4.0   
1900-01-01 21:00:00      3.0    1.0     7.0      NaN    1.0    11.0      1.0   
1900-01-01 22:00:00      NaN    1.0     6.0      2.0    NaN    17.0      1.0   
1900-01-01 23:00:00      1.0    NaN     NaN      1.0    NaN    13.0      NaN   
1900-01-02 00:00:00      NaN    1.0     NaN      NaN    2.0     2.0      2.0   

PURP                                
Day                 Sunday Weekday  
NearestHour                         
1900-01-01 00:00:00    NaN     2.0  
1900-01-01 01:00:00    NaN     NaN  
1900-01-01 02:00:00    NaN     2.0  
1900-01-01 04:00:00    NaN     1.0  
1900-01-01 05:00:00    1.0     2.0  
1900-01-01 06:00:00    1.0    10.0  
1900-01-01 07:00:00    2.0    17.0  
1900-01-01 08:00:00    6.0    78.0  
1900-01-01 09:00:00   10.0    74.0  
1900-01-01 10:00:00   14.0   112.0  
1900-01-01 11:00:00   11.0    83.0  
1900-01-01 12:00:00   29.0   110.0  
1900-01-01 13:00:00   21.0    86.0  
1900-01-01 14:00:00   26.0   109.0  
1900-01-01 15:00:00   23.0    97.0  
1900-01-01 16:00:00   15.0   124.0  
1900-01-01 17:00:00   11.0   109.0  
1900-01-01 18:00:00    4.0   115.0  
1900-01-01 19:00:00    8.0    47.0  
1900-01-01 20:00:00    2.0    45.0  
1900-01-01 21:00:00    4.0    17.0  
1900-01-01 22:00:00    1.0    12.0  
1900-01-01 23:00:00    NaN     2.0  
1900-01-02 00:00:00    NaN     1.0

In [44]:
# National Average Walk Trip Distance 
avg_dist_nat_walk = national_walk.groupby(['NearestHour','PURP'])['TripDist','WTTRDFIN'].mean()
avg_dist_nat_walk = pd.DataFrame(avg_dist_nat_walk)
avg_dist_nat_walk.reset_index()

avg_dist_nat_walk['AvgDist'] = avg_dist_nat_walk['TripDist']/avg_dist_nat_walk['WTTRDFIN']

avg_dist_nat_walk.pivot_table(index = ['NearestHour'], values = 'AvgDist', columns = 'PURP')

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


PURP,HBO,HBW,NHB
NearestHour,,,
1900-01-01 00:00:00,0.665372,1.140169,0.254314
1900-01-01 01:00:00,0.331720,1.818587,3.941451
1900-01-01 02:00:00,0.897576,0.645482,1.410305
1900-01-01 03:00:00,0.356723,0.381000,0.505852
1900-01-01 04:00:00,2.420295,0.949275,0.641367
1900-01-01 05:00:00,0.855320,0.706411,0.892974
1900-01-01 06:00:00,1.067777,1.078918,0.769166
1900-01-01 07:00:00,0.764258,0.701864,12.139362
1900-01-01 08:00:00,1.383872,1.675016,1.040766


In [46]:
# National Average Walk Trip Distance 
avg_dist_nat_walk = national_walk.groupby(['Day','NearestHour','PURP'])['TripDist','WTTRDFIN'].mean()
avg_dist_nat_walk = pd.DataFrame(avg_dist_nat_walk)
avg_dist_nat_walk.reset_index()

avg_dist_nat_walk['AvgDist'] = avg_dist_nat_walk['TripDist']/avg_dist_nat_walk['WTTRDFIN']

avg_dist_nat_walk.pivot_table(index = ['NearestHour'], values = 'AvgDist', columns = ['PURP','Day'])

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


PURP                      HBO                           HBW            \
Day                  Saturday    Sunday   Weekday  Saturday    Sunday   
NearestHour                                                             
1900-01-01 00:00:00  0.090373       NaN  0.708873  0.075000  0.698043   
1900-01-01 01:00:00  0.267000       NaN  0.408410  1.313000       NaN   
1900-01-01 02:00:00       NaN  0.781000  0.934328       NaN       NaN   
1900-01-01 03:00:00  0.250000       NaN  0.390000       NaN       NaN   
1900-01-01 04:00:00  4.046645  1.067631  2.071736  0.625000  0.311000   
1900-01-01 05:00:00  2.877188  1.646098  0.747079       NaN  0.100000   
1900-01-01 06:00:00  1.521059  1.107732  1.031002  0.616040  1.579954   
1900-01-01 07:00:00  1.536646  1.034407  0.708100  0.721508  0.396145   
1900-01-01 08:00:00  0.891315  1.511753  1.404954  0.581991  0.503005   
1900-01-01 09:00:00  0.484214  0.575772  1.037778  0.468562  0.094721   
1900-01-01 10:00:00  0.778938  1.067330  0.919292  0.467272  0.648820   
1900-01-01 11:00:00  0.873388  0.995384  0.864628  0.535489  0.128236   
1900-01-01 12:00:00  0.745016  0.701507  0.903725  0.326855  0.512236   
1900-01-01 13:00:00  0.608546  0.735992  0.765333  0.620416  0.089120   
1900-01-01 14:00:00  1.048390  0.645651  0.769605  0.414292  1.536836   
1900-01-01 15:00:00  1.015534  1.076590  0.583289  0.211989  1.466964   
1900-01-01 16:00:00  1.050459  1.125859  0.585600  1.446424  0.758480   
1900-01-01 17:00:00  0.616883  0.740713  0.665529  0.327624  0.424536   
1900-01-01 18:00:00  0.162752  0.705869  0.979421  0.983794  0.437215   
1900-01-01 19:00:00  0.756271  0.676837  0.925604  0.563294  0.462132   
1900-01-01 20:00:00  0.799090  0.809947  1.078801  0.616591  0.410895   
1900-01-01 21:00:00  0.650917  1.739953  0.330737  0.478312  0.304071   
1900-01-01 22:00:00  0.955789  0.505922  0.485897  0.871391  0.789330   
1900-01-01 23:00:00  0.605824  0.458833  0.486439  0.621406  0.200677   
1900-01-02 00:00:00  0.214790  1.048000  1.098013  1.111000  1.710652   

PURP                                 NHB                       
Day                    Weekday  Saturday    Sunday    Weekday  
NearestHour                                                    
1900-01-01 00:00:00   1.298777  0.227738  0.111699   0.321391  
1900-01-01 01:00:00   2.480909  0.225625       NaN   4.135633  
1900-01-01 02:00:00   0.645482  0.207291  0.192000   1.909121  
1900-01-01 03:00:00   0.381000  0.484000       NaN   0.889000  
1900-01-01 04:00:00   2.450435  0.689820  1.000000   0.376571  
1900-01-01 05:00:00   0.731210  0.673043  1.584925   0.833848  
1900-01-01 06:00:00   1.080410  0.465534  0.948983   0.786598  
1900-01-01 07:00:00   0.774597  2.785249  0.688175  13.668857  
1900-01-01 08:00:00   1.776786  0.798824  1.100155   1.059193  
1900-01-01 09:00:00   0.568389  0.444594  0.668413   0.554153  
1900-01-01 10:00:00   0.669705  0.518185  0.401266   1.934476  
1900-01-01 11:00:00   0.803793  0.436315  0.399211   0.700729  
1900-01-01 12:00:00   0.507266  0.477559  0.410986   0.513218  
1900-01-01 13:00:00   0.327276  0.481803  0.839491   0.442069  
1900-01-01 14:00:00   0.849274  0.404462  0.452855   0.633138  
1900-01-01 15:00:00   0.709195  0.427966  0.384609   0.498549  
1900-01-01 16:00:00   3.788190  0.756736  0.541556   0.580652  
1900-01-01 17:00:00   0.724118  0.433248  0.719301   1.039179  
1900-01-01 18:00:00   2.410487  0.698892  0.629193   0.506109  
1900-01-01 19:00:00   0.770967  0.280441  0.503130   0.433743  
1900-01-01 20:00:00  10.030462  0.372004  0.389375   0.713945  
1900-01-01 21:00:00   0.682515  0.215610  0.625194   0.464706  
1900-01-01 22:00:00   0.745452  0.506972  0.600470   0.417792  
1900-01-01 23:00:00   0.570930  0.147845  0.035771   0.384847  
1900-01-02 00:00:00   0.937412  0.298505  0.329862   0.613337

In [50]:
# National Average Walk Trip Distance 
avg_dist_nat_walk = national_walk.groupby(['Day','NearestHour','PURP'])['WTTRDFIN'].sum()
avg_dist_nat_walk = pd.DataFrame(avg_dist_nat_walk)
avg_dist_nat_walk.reset_index()

avg_dist_nat_walk['WeightedTrips'] = avg_dist_nat_walk['WTTRDFIN']

avg_dist_nat_walk.pivot_table(index = ['NearestHour'], values = 'WeightedTrips', columns = ['PURP','Day'])

PURP                          HBO                                       HBW  \
Day                      Saturday        Sunday       Weekday      Saturday   
NearestHour                                                                   
1900-01-01 00:00:00  3.559564e+05           NaN  4.705035e+06  5.782818e+05   
1900-01-01 01:00:00  5.572056e+06           NaN  4.702336e+06  1.061196e+06   
1900-01-01 02:00:00           NaN  1.240814e+05  3.935800e+05           NaN   
1900-01-01 03:00:00  4.197691e+05           NaN  1.346259e+06           NaN   
1900-01-01 04:00:00  1.847227e+06  4.512604e+05  6.867801e+06  1.749524e+06   
1900-01-01 05:00:00  4.400638e+05  5.785949e+06  5.049089e+07           NaN   
1900-01-01 06:00:00  1.974745e+07  1.406281e+07  2.586793e+08  2.322082e+06   
1900-01-01 07:00:00  3.927545e+07  3.676159e+07  7.170370e+08  8.771999e+06   
1900-01-01 08:00:00  8.312170e+07  7.996311e+07  1.457017e+09  8.330407e+06   
1900-01-01 09:00:00  4.668920e+07  7.427098e+07  4.672718e+08  4.651530e+06   
1900-01-01 10:00:00  7.646049e+07  1.245306e+08  4.017429e+08  1.452373e+07   
1900-01-01 11:00:00  7.162634e+07  9.846446e+07  2.463327e+08  2.299322e+06   
1900-01-01 12:00:00  8.952396e+07  1.164830e+08  3.654545e+08  1.058669e+07   
1900-01-01 13:00:00  3.755196e+07  4.848764e+07  2.699543e+08  6.552583e+06   
1900-01-01 14:00:00  8.923196e+07  8.580213e+07  5.488307e+08  3.532528e+06   
1900-01-01 15:00:00  5.706344e+07  5.225089e+07  8.954296e+08  1.817676e+06   
1900-01-01 16:00:00  1.011117e+08  7.168585e+07  8.681693e+08  1.455503e+07   
1900-01-01 17:00:00  5.393836e+07  5.345494e+07  4.774805e+08  1.429398e+07   
1900-01-01 18:00:00  8.379510e+07  9.723232e+07  6.146469e+08  1.278298e+07   
1900-01-01 19:00:00  5.310117e+07  2.938535e+07  3.353668e+08  4.146711e+06   
1900-01-01 20:00:00  6.042732e+07  7.864577e+07  3.331535e+08  9.903459e+06   
1900-01-01 21:00:00  4.628515e+07  1.945301e+07  1.594958e+08  3.941029e+06   
1900-01-01 22:00:00  2.006700e+07  2.450608e+07  1.225187e+08  1.041750e+06   
1900-01-01 23:00:00  8.628022e+06  9.519738e+06  5.319473e+07  1.172582e+06   
1900-01-02 00:00:00  2.901924e+06  1.502326e+05  2.214329e+07  1.507325e+06   

PURP                                                      NHB                \
Day                        Sunday       Weekday      Saturday        Sunday   
NearestHour                                                                   
1900-01-01 00:00:00  1.594200e+06  8.327456e+06  7.967175e+06  3.572099e+06   
1900-01-01 01:00:00           NaN  8.100695e+05  1.557100e+06           NaN   
1900-01-01 02:00:00           NaN  1.035610e+06  1.387958e+06  6.086184e+05   
1900-01-01 03:00:00           NaN  3.291219e+05  3.011668e+06           NaN   
1900-01-01 04:00:00  5.919489e+04  4.030943e+05  2.734199e+06  5.761988e+06   
1900-01-01 05:00:00  5.899276e+05  1.442558e+07  2.627008e+06  1.634341e+06   
1900-01-01 06:00:00  1.961151e+06  6.185234e+07  7.353660e+06  5.472945e+06   
1900-01-01 07:00:00  1.905309e+07  7.771728e+07  1.656615e+07  5.183503e+06   
1900-01-01 08:00:00  9.446126e+06  1.982536e+08  3.947970e+07  2.255026e+07   
1900-01-01 09:00:00  2.383682e+06  1.492731e+08  6.268164e+07  3.293974e+07   
1900-01-01 10:00:00  6.726852e+05  1.102494e+08  1.392454e+08  8.679835e+07   
1900-01-01 11:00:00  1.053376e+07  4.466180e+07  9.765675e+07  8.055868e+07   
1900-01-01 12:00:00  7.370695e+06  6.621687e+07  1.738339e+08  1.534003e+08   
1900-01-01 13:00:00  6.872530e+06  7.434472e+07  1.407532e+08  1.071376e+08   
1900-01-01 14:00:00  1.132460e+07  1.232566e+08  1.305825e+08  1.196943e+08   
1900-01-01 15:00:00  7.447783e+06  5.124726e+07  1.527254e+08  1.280796e+08   
1900-01-01 16:00:00  5.438405e+06  1.481210e+08  1.207227e+08  1.071348e+08   
1900-01-01 17:00:00  6.773193e+06  1.440743e+08  7.614827e+07  4.760806e+07   
1900-01-01 18:00:00  3.224204e+06  1.239336e+08  6.554532e+07  7.102561e+07   
1900-01-01 19:00:00  2.770548e+06  6.233157e+07  

In [52]:
# National Average Walk Trip Distance 
avg_dist_nat_walk = national_walk.groupby(['Day','NearestHour','PURP']).size().reset_index(name='counts')
avg_dist_nat_walk = pd.DataFrame(avg_dist_nat_walk)
avg_dist_nat_walk.reset_index()

avg_dist_nat_walk['UNWeightedTrips'] = avg_dist_nat_walk['counts']

avg_dist_nat_walk.pivot_table(index = ['NearestHour'], values = 'UNWeightedTrips', columns = ['PURP','Day'])

PURP                     HBO                     HBW                     NHB  \
Day                 Saturday Sunday Weekday Saturday Sunday Weekday Saturday   
NearestHour                                                                    
1900-01-01 00:00:00      2.0    NaN    19.0      1.0    2.0     9.0     10.0   
1900-01-01 01:00:00      1.0    NaN    11.0      1.0    NaN     4.0      5.0   
1900-01-01 02:00:00      NaN    1.0     5.0      NaN    NaN     4.0      6.0   
1900-01-01 03:00:00      1.0    NaN     1.0      NaN    NaN     1.0      1.0   
1900-01-01 04:00:00      6.0    7.0    48.0      1.0    1.0     4.0      2.0   
1900-01-01 05:00:00      4.0   11.0   141.0      NaN    1.0    21.0      5.0   
1900-01-01 06:00:00     56.0   53.0   722.0      6.0    5.0   108.0     18.0   
1900-01-01 07:00:00     82.0   96.0  1369.0      9.0    7.0   172.0     27.0   
1900-01-01 08:00:00    167.0  172.0  2741.0     16.0   17.0   393.0     87.0   
1900-01-01 09:00:00    116.0  137.0  1061.0      8.0    6.0   208.0    112.0   
1900-01-01 10:00:00    192.0  217.0  1079.0     21.0    4.0   183.0    209.0   
1900-01-01 11:00:00     93.0  129.0   552.0     10.0    3.0    78.0    206.0   
1900-01-01 12:00:00    157.0  152.0   851.0      9.0    5.0   172.0    310.0   
1900-01-01 13:00:00     74.0   93.0   624.0     10.0    4.0   133.0    238.0   
1900-01-01 14:00:00    155.0  148.0  1244.0     12.0    7.0   171.0    298.0   
1900-01-01 15:00:00     97.0  103.0  1643.0     12.0    9.0   115.0    249.0   
1900-01-01 16:00:00    164.0  183.0  1871.0     17.0   12.0   271.0    244.0   
1900-01-01 17:00:00     97.0   98.0   972.0     12.0    8.0   245.0    129.0   
1900-01-01 18:00:00    128.0  155.0  1462.0      8.0   12.0   244.0    165.0   
1900-01-01 19:00:00    102.0   78.0   900.0      6.0    4.0    91.0    117.0   
1900-01-01 20:00:00     89.0   96.0   941.0      6.0    9.0    81.0    111.0   
1900-01-01 21:00:00     47.0   43.0   349.0      8.0    6.0    40.0     52.0   
1900-01-01 22:00:00     44.0   40.0   340.0      5.0    9.0    49.0     76.0   
1900-01-01 23:00:00     15.0   16.0   102.0      4.0    2.0    18.0     12.0   
1900-01-02 00:00:00      5.0    1.0    43.0      1.0    4.0     9.0      9.0   

PURP                                
Day                 Sunday Weekday  
NearestHour                         
1900-01-01 00:00:00    3.0    18.0  
1900-01-01 01:00:00    NaN    14.0  
1900-01-01 02:00:00    2.0    10.0  
1900-01-01 03:00:00    NaN     1.0  
1900-01-01 04:00:00    1.0     4.0  
1900-01-01 05:00:00    3.0    36.0  
1900-01-01 06:00:00   15.0   165.0  
1900-01-01 07:00:00   24.0   371.0  
1900-01-01 08:00:00   55.0  1021.0  
1900-01-01 09:00:00   68.0   829.0  
1900-01-01 10:00:00  159.0  1410.0  
1900-01-01 11:00:00  147.0  1228.0  
1900-01-01 12:00:00  265.0  3248.0  
1900-01-01 13:00:00  180.0  2367.0  
1900-01-01 14:00:00  249.0  2134.0  
1900-01-01 15:00:00  156.0  1412.0  
1900-01-01 16:00:00  187.0  1525.0  
1900-01-01 17:00:00  115.0   971.0  
1900-01-01 18:00:00  123.0  1145.0  
1900-01-01 19:00:00   73.0   695.0  
1900-01-01 20:00:00   67.0   634.0  
1900-01-01 21:00:00   33.0   266.0  
1900-01-01 22:00:00   20.0   207.0  
1900-01-01 23:00:00    2.0    75.0  
1900-01-02 00:00:00    3.0    15.0

In [8]:
# FLORIDA

In [9]:
fl_bike = df.loc[(df["TRPTRANS"] == 2) & (df["HHSTATE"] == "FL"),]
fl_walk = df.loc[(df["TRPTRANS"] == 1) & (df["HHSTATE"] == "FL"),]

In [10]:
fl_bike.shape


(116, 11)

In [11]:
fl_walk.shape

(812, 11)

In [12]:
# Average Bike Trip Distance in Florida

avg_dist_fl_bike = fl_bike.groupby('Day')['TripDist','WTTRDFIN'].sum()
avg_dist_fl_bike = pd.DataFrame(avg_dist_fl_bike)

avg_dist_fl_bike['AvgDist'] = avg_dist_fl_bike['TripDist']/avg_dist_fl_bike['WTTRDFIN']
avg_dist_fl_bike

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,TripDist,WTTRDFIN,AvgDist
Day,,,
Saturday,4.906882e+07,2.752865e+07,1.782464
Sunday,1.252354e+08,3.786933e+07,3.307041
Weekday,4.052103e+08,1.769850e+08,2.289517


In [13]:
#Average Walk Trip Distance in Florida

avg_dist_fl_walk = fl_walk.groupby('Day')['TripDist','WTTRDFIN'].sum()
avg_dist_fl_walk = pd.DataFrame(avg_dist_fl_walk)
avg_dist_fl_walk

avg_dist_fl_walk['AvgDist'] = avg_dist_fl_walk['TripDist']/avg_dist_fl_walk['WTTRDFIN']
avg_dist_fl_walk

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,TripDist,WTTRDFIN,AvgDist
Day,,,
Saturday,1.038046e+08,1.989933e+08,0.521649
Sunday,1.333528e+08,2.905080e+08,0.459033
Weekday,2.423820e+09,1.185929e+09,2.043815


In [14]:
#Average Bike Trip Distance in Florida
avg_dist_fl_bike = fl_bike.groupby(['Day','TRIPPURP'])['TripDist','WTTRDFIN'].sum()
avg_dist_fl_bike = pd.DataFrame(avg_dist_fl_bike)
avg_dist_fl_bike.reset_index()

avg_dist_fl_bike['AvgDist'] = avg_dist_fl_bike['TripDist']/avg_dist_fl_bike['WTTRDFIN']

avg_dist_fl_bike.pivot_table(index = ['Day'], values = 'AvgDist', columns = 'TRIPPURP')


C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


TRIPPURP,HBO,HBSHOP,HBSOCREC,HBW,NHB
Day,,,,,
Saturday,2.299462,0.323000,1.660445,NaN,2.828000
Sunday,7.158275,0.636652,4.000844,6.359000,NaN
Weekday,1.916611,0.878860,1.868723,5.242045,1.449847


In [15]:
# Florida - Average Walk Trip Distance 
avg_dist_fl_walk = fl_walk.groupby(['Day','TRIPPURP'])['TripDist','WTTRDFIN'].mean()
avg_dist_fl_walk = pd.DataFrame(avg_dist_fl_walk)
avg_dist_fl_walk.reset_index()

avg_dist_fl_walk['AvgDist'] = avg_dist_fl_walk['TripDist']/avg_dist_fl_walk['WTTRDFIN']

avg_dist_fl_walk.pivot_table(index = ['Day'], values = 'AvgDist', columns = 'TRIPPURP')

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


TRIPPURP,HBO,HBSHOP,HBSOCREC,HBW,NHB
Day,,,,,
Saturday,0.785272,0.604649,0.552377,0.236353,0.242131
Sunday,0.518640,0.315484,0.439120,0.258000,0.539781
Weekday,0.893879,5.220103,0.691502,26.116039,0.539827


In [17]:
fl_cbsa_bike = df.loc[(df["TRPTRANS"] == 2) & (df["HH_CBSA"] == '36740'),]
fl_cbsa_walk = df.loc[(df["TRPTRANS"] == 1) & (df["HH_CBSA"] == '36740'),]

In [18]:
fl_cbsa_bike.shape

(16, 11)

In [19]:
fl_cbsa_walk.shape

(75, 11)

In [20]:
########### CBSA ############

In [21]:
# FL CBSA - Average Bike Trip Distance

avg_dist_fl_bike = fl_cbsa_bike.groupby('Day')['TripDist','WTTRDFIN'].sum()
avg_dist_fl_bike = pd.DataFrame(avg_dist_fl_bike)

avg_dist_fl_bike['AvgDist'] = avg_dist_fl_bike['TripDist']/avg_dist_fl_bike['WTTRDFIN']
avg_dist_fl_bike

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,TripDist,WTTRDFIN,AvgDist
Day,,,
Weekday,2.748425e+07,4.538354e+07,0.6056


In [22]:
# FL CBSA - Average Walk Trip Distance

avg_dist_fl_walk = fl_cbsa_walk.groupby('Day')['TripDist','WTTRDFIN'].sum()
avg_dist_fl_walk = pd.DataFrame(avg_dist_fl_walk)
avg_dist_fl_walk

avg_dist_fl_walk['AvgDist'] = avg_dist_fl_walk['TripDist']/avg_dist_fl_walk['WTTRDFIN']
avg_dist_fl_walk

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,TripDist,WTTRDFIN,AvgDist
Day,,,
Saturday,4.152402e+07,4.482420e+07,0.926375
Sunday,1.784226e+06,1.227661e+07,0.145335
Weekday,5.315958e+07,8.403644e+07,0.632578


In [23]:
# FL CBSA - Average Bike Trip Distance
avg_dist_fl_bike = fl_cbsa_bike.groupby(['Day','TRIPPURP'])['TripDist','WTTRDFIN'].sum()
avg_dist_fl_bike = pd.DataFrame(avg_dist_fl_bike)
avg_dist_fl_bike.reset_index()

avg_dist_fl_bike['AvgDist'] = avg_dist_fl_bike['TripDist']/avg_dist_fl_bike['WTTRDFIN']

avg_dist_fl_bike.pivot_table(index = ['Day'], values = 'AvgDist', columns = 'TRIPPURP')


C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


TRIPPURP,HBO,HBSHOP,HBSOCREC
Day,,,
Weekday,0.565725,0.570647,1.071965


In [24]:
# FL CBSA - Average Walk Trip Distance 
avg_dist_fl_walk = fl_cbsa_walk.groupby(['Day','TRIPPURP'])['TripDist','WTTRDFIN'].mean()
avg_dist_fl_walk = pd.DataFrame(avg_dist_fl_walk)
avg_dist_fl_walk.reset_index()

avg_dist_fl_walk['AvgDist'] = avg_dist_fl_walk['TripDist']/avg_dist_fl_walk['WTTRDFIN']

avg_dist_fl_walk.pivot_table(index = ['Day'], values = 'AvgDist', columns = 'TRIPPURP')

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


TRIPPURP,HBO,HBSHOP,HBSOCREC,NHB
Day,,,,
Saturday,1.109104,1.4795,0.636407,0.219000
Sunday,NaN,NaN,0.087854,0.295250
Weekday,1.108155,1.0270,0.300567,0.362885
